In [1]:
import pandas as pd
import requests
import json
from datetime import datetime

In [ ]:
cities = ["Frankfurt", "Berlin", "Cologne", "Munich", "Hamburg"]

In [90]:
def weather_1(cities):
    from API_keys import open_weather_API_key as API_key

    now = datetime.now()

    weather_dict1= {"City": [],
                "Country": [],
                "Forecast_Time": [],
                "Weather_Outlook": [],
                "Weather_Detailed": [],
                "Temperature": [],
                "Real_Feel_Temperature": [],
                "Wind_Speed": [],
                "Rain_Risk": [],
                "Rain_fall_mm": [],
                "Snow_fall_mm": [],
                "info_retrieved_at": []}
    
    for city in cities:
        url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric"
        response = requests.get(url)
        wdj_response = response.json()

        for wdj in  wdj_response['list'][:8]:
                weather_dict1['City'].append(wdj_response['city']['name']),
                weather_dict1['Country'].append(wdj_response['city']['country']),
                weather_dict1['Forecast_Time'].append(wdj['dt_txt']),
                weather_dict1['Weather_Outlook'].append(wdj['weather'][0]['main']),
                weather_dict1['Weather_Detailed'].append(wdj['weather'][0]['description']),
                weather_dict1['Temperature'].append(wdj['main']['temp']),
                weather_dict1['Real_Feel_Temperature'].append(wdj['main']['feels_like']),
                weather_dict1['Wind_Speed'].append(wdj['wind']['speed']),
                weather_dict1['Rain_Risk'].append(wdj['pop'])

                try:
                  weather_dict1["Rain_fall_mm"].append(wdj["rain"]["3h"])
                except:
                  weather_dict1["Rain_fall_mm"].append("0")
                try:
                  weather_dict1["Snow_fall_mm"].append(wdj["snow"]["3h"])
                except:
                  weather_dict1["Snow_fall_mm"].append("0")

                weather_dict1["info_retrieved_at"].append(now.strftime("%d/%m/%Y %H:%M:%S"))


    return pd.DataFrame(weather_dict1).pipe(clean_weather).pipe(rename_reshape)

In [ ]:
cities = ["Berlin", "Frankfurt", "Cologne", "Munich", "Hamburg"]



weather_1(cities)


In [1]:
def rename_reshape(df):
    df = df.rename(columns={'City':'city_name', 'Country':'country_code'})
    df['city_code']= df['city_name'].str[:3]
    df['city_code'] =df['city_code'].str.upper()
    return df

In [2]:
def clean_weather(df):    
    numeric_columns = ['Temperature', 'Real_Feel_Temperature', 'Wind_Speed', 'Rain_Risk', 'Rain_fall_mm', 'Snow_fall_mm']
    datetime_columns = ['Forecast_Time', 'info_retrieved_at']

    for measure in numeric_columns:
        df[measure] = pd.to_numeric(df[measure])
    for dt in datetime_columns:
        df[dt] = pd.to_datetime(df[dt])
    return df

In [85]:
import sqlalchemy
import pymysql
import numpy as np

In [88]:
from password_sql import con

In [87]:
weather_df.to_sql('weather',
              if_exists='append',
              con=con,
              index=False)

64